<a href="https://colab.research.google.com/github/Senhuh/Reddimood/blob/Model-Training/Model_Training_Sentiment_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

uploaded = files.upload()
df = pd.read_csv("train.csv", encoding='unicode_escape')
df.head()

Saving train.csv to train (3).csv


,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


### Data Preprocessing

In [ ]:
df.shape

(27481, 10)

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.isnull().sum()

,0
textID,0
text,1
selected_text,1
sentiment,0
Time of Tweet,0
Age of User,0
Country,0
Population -2020,0
Land Area (Km²),0
Density (P/Km²),0


In [ ]:
df.dropna(inplace=True)

In [ ]:
df = df.drop(df.iloc[:,4:10], axis = 1)
df = df.drop('textID', axis = 1)
df['Post'] = df[['text', 'selected_text']].agg(' '.join, axis=1)
df = df.drop(df.iloc[:,0:2], axis = 1)
df.head()

,sentiment,Post
0,neutral,"I`d have responded, if I were going I`d have ..."
1,negative,Sooo SAD I will miss you here in San Diego!!!...
2,negative,my boss is bullying me... bullying me
3,negative,what interview! leave me alone leave me alone
4,negative,"Sons of ****, why couldn`t they put them on t..."


In [ ]:
df.shape

(27480, 2)

In [ ]:
df = df.replace({"positive":1, "negative": -1, "neutral": 0})
df.head(15)

<ipython-input-325-3e583be2f724>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"positive":1, "negative": -1, "neutral": 0})


,sentiment,Post
0,0,"I`d have responded, if I were going I`d have ..."
1,-1,Sooo SAD I will miss you here in San Diego!!!...
2,-1,my boss is bullying me... bullying me
3,-1,what interview! leave me alone leave me alone
4,-1,"Sons of ****, why couldn`t they put them on t..."
5,0,http://www.dothebouncy.com/smf - some shameles...
6,1,2am feedings for the baby are fun when he is a...
7,0,Soooo high Soooo high
8,0,Both of you Both of you
9,1,Journey!? Wow... u just became cooler. hehe....


### Text Preprocessing

In [ ]:
import re
import string

In [ ]:
#convert uppercae to lowercase
df['Post'] = df['Post'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#remove links
df['Post'] = df['Post'].apply(lambda x: " ".join(re.sub("https?:\/\/.*[\r\n]*","",x, flags=re.MULTILINE) for x in x.split()))

In [ ]:
#remove punctuations

def remove_punctuations(text):
  for punctuation in string.punctuation:
    text=text.replace(punctuation, '')
  return text

df["Post"] = df["Post"].apply(remove_punctuations)

In [ ]:
#remove numbers

df["Post"] = df["Post"].str.replace('\d+', '', regex = True)


In [ ]:
df.head(15)

,sentiment,Post
0,0,id have responded if i were going id have resp...
1,-1,sooo sad i will miss you here in san diego soo...
2,-1,my boss is bullying me bullying me
3,-1,what interview leave me alone leave me alone
4,-1,sons of why couldnt they put them on the rele...
5,0,some shameless plugging for the best rangers...
6,1,am feedings for the baby are fun when he is al...
7,0,soooo high soooo high
8,0,both of you both of you
9,1,journey wow u just became cooler hehe is that ...


In [ ]:
pip install nltk

In [ ]:
#remove stopwords
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
sw = stopwords.words('english')
df['Post'] = df['Post'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()
df["Post"] = df["Post"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))

In [ ]:
df.head(5)

,sentiment,Post
0,0,id respond go id respond go
1,-1,sooo sad miss san diego sooo sad
2,-1,boss bulli bulli
3,-1,interview leav alon leav alon
4,-1,son couldnt put releas alreadi bought son


In [ ]:
df.isnull().sum()

,0
sentiment,0
Post,0


### Model Building

In [ ]:
pip install vaderSentiment

In [ ]:
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import vaderSentiment.vaderSentiment as vv


In [ ]:
score = SentimentIntensityAnalyzer()
help(score)

Help on SentimentIntensityAnalyzer in module vaderSentiment.vaderSentiment object:

class SentimentIntensityAnalyzer(builtins.object)
 |  SentimentIntensityAnalyzer(lexicon_file='vader_lexicon.txt', emoji_lexicon='emoji_utf8_lexicon.txt')
 |  
 |  Give a sentiment intensity score to sentences.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, lexicon_file='vader_lexicon.txt', emoji_lexicon='emoji_utf8_lexicon.txt')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  make_emoji_dict(self)
 |      Convert emoji lexicon file to a dictionary
 |  
 |  make_lex_dict(self)
 |      Convert lexicon file to a dictionary
 |  
 |  polarity_scores(self, text)
 |      Return a float for sentiment strength based on the input text.
 |      Positive values are positive valence, negative value are negative
 |      valence.
 |  
 |  score_valence(self, sentiments, text)
 |  
 |  sentiment_valence(self, valence, sentitext, item, i, sentiments)
 |  
 |  --------------------

In [ ]:
text = df["Post"]

results = []

for line in text:
  scores = score.polarity_scores(line)
  scores['text'] = line
  results.append(scores)


In [ ]:
new_df = pd.DataFrame.from_records(results)
new_df.compound = new_df.compound.round()
new_df.head()

,neg,neu,pos,compound,text
0,0.000,1.000,0.0,0.0,"I`d have responded, if I were going I`d have ..."
1,0.530,0.470,0.0,-1.0,Sooo SAD I will miss you here in San Diego!!!...
2,0.609,0.391,0.0,-1.0,my boss is bullying me... bullying me
3,0.626,0.374,0.0,-1.0,what interview! leave me alone leave me alone
4,0.000,1.000,0.0,0.0,"Sons of ****, why couldn`t they put them on t..."


In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
score = accuracy_score(df['sentiment'], new_df['compound'])

In [ ]:
print(score)

0.6377001455604075


In [ ]:
df.head(10)

,sentiment,Post
0,0,id respond go id respond go
1,-1,sooo sad miss san diego sooo sad
2,-1,boss bulli bulli
3,-1,interview leav alon leav alon
4,-1,son couldnt put releas alreadi bought son
5,0,shameless plug best ranger forum earth shamele...
6,1,feed babi fun smile coo fun
7,0,soooo high soooo high
8,0,
9,1,journey wow u becam cooler hehe possibl wow u ...


In [ ]:
#most likely hindi na needed(this + codes below it), pero dito muna jic
import praw
import pandas as pd

#initializing reddit instance
user_agent = "scraper"
reddit = praw.Reddit(
    client_id="F5KdetPuX-XUUVJSKn7b1A",
    client_secret = "VMtZzO_oYfzFVuLN2ju_4u_KhrLb0w",
    user_agent = user_agent
)

#subreddit to be scraped
subreddit = reddit.subreddit('RandomThoughts')

#Define lists to store data
data = []

#scraping posts
for post in subreddit.hot(limit=None):
    data.append({
        'Title': post.title,
        'Text': post.selftext,
        'Text_only': post.is_self,
        'Ratio': post.upvote_ratio
       })

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [ ]:
# Create pandas DataFrame for reddit posts
df = pd.DataFrame(data)
df.head()

,Title,Text,Text_only,Ratio
0,Saturday Morning Political Cartoon Thread,"It's Saturday, folks. Let's all kick back with...",True,0.90
1,Super Bowl champion Philadelphia Eagles skippi...,,False,0.98
2,Protesters at Republican Event Told 'Your Voic...,,False,0.96
3,"James Carville predicts Trump, GOP are in ‘mid...",,False,0.89
4,Update on town hall chaos: Woman who was dragg...,,False,0.99


In [ ]:
#save dataframe
df.to_csv('reddit_data.csv')

In [ ]:
#check current dataset state
df.head()

,Title,Text,Text_only
0,Changes to r/RandomThoughts are coming,"Hey friends, Just informing all of you that we...",True
1,Pretend it's Monday...I didn't lie 😅,We have plans in place to begin using r/random...,True
2,Do you think you are AGING like FINE WINE?,,True
3,"Imagine if sedation never existed-just raw, un...",,True
4,"I hate the term ""affordable housing""",I really hate hearing about affordable housing...,True


The Text_only is a PRAW attribute that tells if a reddit post is joined with other multi-media attachments or mainly text-only. By checking posts with a text_only value of FALSE, a cleaner representation of data can be made.


In [ ]:
print(df[df['Text_only']=="FALSE"])

Empty DataFrame
Columns: [Title, Text, Text_only]
Index: []


In [ ]:
df.isnull().sum()

,0
Title,0
Text,0
Text_only,0
Post,0


In [ ]:
df['Post'] = df[['Title', 'Text']].agg(' '.join, axis=1)
df.head()
df.to_csv('reddit_data2.csv')